In [52]:

import collections
import json
import re

import spacy

In [53]:
nlp = spacy.load("en_core_web_lg")
MINLEN = 100

In [54]:
with open("headers-100000.json") as f:
    page = json.load(f)

In [55]:
def getlead(text, patt=re.compile("==* .* ==*")):
    patt = re.compile("==* .* ==*")
    lead = re.split(patt, text)[0]
    return [(str(t).lower(), t.pos_, t.ent_type_) for t in nlp(lead) if str(t).strip()]

def rawlead(text, patt=re.compile("==* .* ==*")):
    patt = re.compile("==* .* ==*")
    lead = re.split(patt, text)[0]
    return lead.strip().replace("–", "-")

def projection(
        s, popular, 
        numbers=set("1234567890"), 
        ents={# "DATE", "CARDINAL", "ORDINAL", "TIME"
              "ORG", "GPE", "NORP", "FAC", "LOC", "PRODUCT", 
              "EVENT", "LANGUAGE", "MONEY", "PERCENT", "QUANTITY"}):
    w, pos, ent_type = s
    if pos == "NUM":
        return ent_type if ent_type in ents else pos
        # return pos
    elif not numbers.isdisjoint(w):
        return ent_type if ent_type in ents else pos
        # return pos
    elif (w.lower(), pos) in popular:
        return w.lower()
    else:
        return ent_type if ent_type in ents else pos
        # return pos

In [57]:
keys = sorted(page)
lead = []
for i, k in enumerate(keys):
    if len(lead) > i:
        continue
    lead.append(rawlead(page[k]))
    if i % 25 == 0:
        print("\r", i, end="")
counts = collections.Counter(i for l in lead for i in l)
common = set([chr(i) for i in range(256)])
sublead = ["".join(i for i in l if i in common) for l in lead]
with open("nice-char.json", "w") as f:
    json.dump(sublead, f)

 9992573825

In [ ]:
keys = sorted(page)
lead = []
for i, k in enumerate(keys):
    if len(lead) > i:
        continue
    lead.append(getlead(page[k]))
    if i % 25 == 0:
        print("\r", i, end="")

 71500

In [271]:
sublead = [p for p in lead if len(p) > MINLEN]
counts = collections.Counter((w, pos) for l in sublead for (w, pos, _) in l)
popular = set(p for p, c in counts.most_common(6000))
cleaned = [[projection(point, popular) for point in p] for p in sublead]

In [284]:
words = sorted(set(w for p in cleaned for w in p)) + ["STOPPER"]
wids = {w:i for i,w in enumerate(words)}

In [287]:
tokens = [[wids[w] for w in l]+[wids["STOPPER"]] for l in cleaned]

In [290]:
tokens = [t for l in tokens for t in l]

In [296]:
sentences = [tokens[i:i+150] for i in range(0, len(tokens)-150, 45)]

In [309]:
with open("numbers.data", "w") as f:
    for s in sentences:
        f.write(" ".join(str(i) for i in s) + "\n")

In [306]:
with open("word-ids.json", "w") as f:
    json.dump(words, f)

In [308]:
len(sentences)

204285

In [320]:
words.pop()

'STOPPER'

In [332]:
data = "352 3492 4562 4237 3083 661 711 3993 652 185 2531 1089 426 88 604 3961 4414 3560 773 2890 1880 833 3791 2678 780 3451 2388 2866 3669 3003 3685 311 4492 2287 4344 4656 4679 408 1404 2618 2409 748 1772 4289 2647 4118 4382 4298 1462 1014 1581 2252 304 3154 3862 3456 2945 2069 548 3215 769 2096 3694 602 973 1741 1135 2448 328 3513 1350 929 800 3398 4673 1976 1932 2565 2739 134 3835 2541 1287 1978 3230 3833 2690 3761 566 1060 3273 637 3447 3747 4387 2071 574 3919 1647 4414 3026 4349 708 2584 1335 3906 1474 2360 605 3385 749 770 330 4651 3756 2023 3541 3758 2000 689 1131 3436 605 575 1648 2997 2705 2374 2840 2217 568 2942 4634 268 4291 3852 987 3705 3301 3368 4379 3441 2866 3643 1133 1894 1236 2675 2072 1930"

[words[int(i)] for i in data.split(" ")]

['assistance',
 'pundit',
 'vegas',
 'sundance',
 'or',
 'called',
 'carol',
 'silent',
 'c.',
 'alaska',
 'liberal',
 'craig',
 'baker',
 'account',
 'british',
 'she',
 'tony',
 'rapidly',
 'chan',
 'n',
 'gradually',
 'church',
 'rose',
 'marked',
 'channel',
 'promoted',
 'kenny',
 'mughal',
 'repertoire',
 'november',
 'reprised',
 'argentine',
 'twin',
 'january',
 'thailand',
 'wba',
 'welsh',
 'average',
 'economist',
 'machine',
 'king',
 'cell',
 'frequently',
 'taken',
 'man',
 'st',
 'threw',
 'tamil',
 'emmy',
 'consider',
 'extreme',
 'island',
 'architect',
 'park',
 'school',
 'prompted',
 'newcastle',
 'hour',
 'bo',
 'perhaps',
 'champion',
 'i',
 'reserve',
 'brit',
 'complications',
 'forward',
 'curtis',
 'landed',
 'arrested',
 'queens',
 'dragon',
 'comics',
 'charting',
 'priest',
 'weeks',
 'health',
 'h.',
 'lived',
 'meeting',
 'admitted',
 'san',
 'light',
 'directly',
 'heart',
 'petersburg',
 'same',
 'martin',
 'robert',
 'bosnia',
 'copa',
 'pitch',
 'bu

In [326]:
len(words)

4814